First time through, following the Fake_News method (https://github.com/georgetown-analytics/From-Russia-With-Love-fake-news-/blob/master/Notebooks/Step_1_Data_Cleaning.ipynb).  

We explore different options in other NBs.

In [1]:
import s3fs
import pandas as pd
pd.set_option('display.max_columns', 100000)
pd.set_option('display.max_row', 1000000)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
# import tldextract """Accurately separate the TLD from the registered domain and subdomains of a URL"""
from tqdm.autonotebook import tqdm
tqdm.pandas(desc="progress-bar", leave=False)
import string
import spacy
from spacy.lang import punctuation
import unicodedata  # might need to pip install unicodedate2 on aws sagemaker
import contractions
from contractions import contractions_dict ## pip installed this
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import STOPWORDS
import warnings
from afinn import Afinn
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set(style='darkgrid',palette='Dark2',rc={'figure.figsize':(9,6),'figure.dpi':90})

punctuation = string.punctuation + '”' + '“' + '–' + '““' + "’’" + '”'
stopword = stopwords.words('english')
stopwords = set(STOPWORDS)
wordnet_lemmatizer = WordNetLemmatizer()

<ipython-input-1-1613962be9f5>:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Download dataset from S3 bucket. 

In [2]:
fs = s3fs.S3FileSystem(anon=False,key='###',secret='###')

genres_df = pd.read_csv('s3://wrangled-1/merged5_genre_df.csv')

In [3]:
genres_df.shape

(86391, 8)

In [4]:
genres_df.keys()

Index(['original_csv', 'artist_name', 'song_name', 'link', 'lyrics',
       'language', 'genre', 'date'],
      dtype='object')

In [5]:
genres_df.loc[genres_df['song_name']=='The Story Of Us']

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
84304,genres_csv,taylor-swift,The Story Of Us,/taylor-swift/the-story-of-us.html,I used to think one day we'd tell the story of...,ENGLISH,Pop,NaN


In [6]:
genres_df.iloc[84304]['lyrics']#genres Taylor Swift The Story of Us

"I used to think one day we'd tell the story of us. How we met and sparks flew instantly. People would say 'They're the lucky ones'. I used to know my place was a spot next to you. Now I'm searching the room for an empty seat. 'Cause lately I don't even know what page you're on. Oh. A simple complication, miscommunication. Has lead to fallout. Too many things that I wish you knew. So many walls up I can't break through. Now I'm standing alone in a crowded room. And we're not speaking. And I'm dying to know. Is it killing you like it's killing me?. And I don't know what to say. Since the twist of fate. When it all broke down. And the story of us looks a lot like a tragedy now. Next chapter. How'd we end up this way?. Se me nervously pulling at my clothes. And trying to look busy. And you're doing your best to avoid me. I'm starting to think one day I'll tell the story of us. How I was losing my mind when I saw you here. But you held your pride like you should've held me. Oh. I'm scared 

Defining functions for first round of cleaning.

In [7]:
# Portions of this are excerpts from Stack Overflow responses
def remove_special_characters(text): 
    """
    Removes special characters from the text document
    """
    # define the pattern to keep. You can check the regex using this url https://regexr.com/
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
    return re.sub(pat, '', text)

def remove_extra_whitespace_tabs(text): 
    """
    Removes extra whitespaces and remove_extra_whitespace_tabs
    """
    #pattern = r'^\s+$|\s+$'
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()

def remove_digits(text): 
    """
    Remove all digits from the text document
     take string input and return a clean text without numbers.
        Use regex to discard the numbers.
    """
    result = ''.join(i for i in text if not i.isdigit()).lower()
    return ' '.join(result.split())

def remove_newlines(text): 
    """
    Remove newline characters from the text document
    """
    return text.replace('\\n', ' ').replace('\\r', ' ').replace('\n', ' ').replace('\r', ' ').replace('\\', ' ')

#normalize to the NFKD (Normalization Form Compatibility Decomposition) form
#that present in the Unicode standard to remain compatible with other encodings
def remove_accented_chars(text): 
    """
    Removes accented characters from the test
    """
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text


import contractions
contractions.fix(genres_df['lyrics'][10])



#expands contractions found in the text
def expand_contractions(text):


    #contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    #def expand_match(contraction):
    #    match = contraction.group(0)
    #    first_char = match[0]
    #    expanded_contraction = contraction_mapping.get(match)\
    #                            if contraction_mapping.get(match)\
    #                            else contraction_mapping.get(match.lower())
    #    expanded_contraction = first_char+expanded_contraction[1:]
    #    return expanded_contraction

    expanded_text = contractions.fix(text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

# replace punctuation characters with spaces
def replace_punctuation(text):
    filters = string.punctuation + '”' + '“' + '–' 
    translate_dict = dict((c, " ") for c in filters)   
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    return text

# Remove stopwords and remove words with 2 or less characters
def stops_letters(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in stopword:
            result.append(token)
            
    return " ".join(result)

#Removes any word that starts with either http or https
def remove_urls(vTEXT):
    #vTEXT = re.sub('http://\S+|https://\S+', '', vTEXT,flags=re.MULTILINE)
    vTEXT = re.sub('http[s]?://\S+', '', vTEXT,flags=re.MULTILINE)
    return(vTEXT)

#Remove words that starts with www
def remove_www(vTEXT):
    vTEXT = re.sub('www\S+', '', vTEXT,flags=re.MULTILINE)
    return(vTEXT)



Convert Lyrics to lowercase

In [8]:
%%time

genres_df['lyrics']=genres_df['lyrics'].apply(lambda x: x.lower())

CPU times: user 160 ms, sys: 42.4 ms, total: 203 ms
Wall time: 202 ms


In [9]:
genres_df.iloc[84304]['lyrics']#genres Taylor Swift The Story of Us

"i used to think one day we'd tell the story of us. how we met and sparks flew instantly. people would say 'they're the lucky ones'. i used to know my place was a spot next to you. now i'm searching the room for an empty seat. 'cause lately i don't even know what page you're on. oh. a simple complication, miscommunication. has lead to fallout. too many things that i wish you knew. so many walls up i can't break through. now i'm standing alone in a crowded room. and we're not speaking. and i'm dying to know. is it killing you like it's killing me?. and i don't know what to say. since the twist of fate. when it all broke down. and the story of us looks a lot like a tragedy now. next chapter. how'd we end up this way?. se me nervously pulling at my clothes. and trying to look busy. and you're doing your best to avoid me. i'm starting to think one day i'll tell the story of us. how i was losing my mind when i saw you here. but you held your pride like you should've held me. oh. i'm scared 

Remove URLs, www, incase they are in the lyrics

In [10]:
%%time

genres_df['lyrics']=genres_df['lyrics'].apply(remove_urls)
genres_df['lyrics']=genres_df['lyrics'].apply(remove_www)

CPU times: user 355 ms, sys: 3.25 ms, total: 358 ms
Wall time: 356 ms


Remove special characters and extra whitespace.

In [11]:
%%time

genres_df['lyrics']=genres_df['lyrics'].apply(remove_special_characters)
genres_df['lyrics'] =genres_df['lyrics'].apply(remove_extra_whitespace_tabs)

CPU times: user 7.82 s, sys: 39.5 ms, total: 7.86 s
Wall time: 7.86 s


In [12]:
genres_df.iloc[84304]['lyrics']#genres Taylor Swift The Story of Us

"i used to think one day we'd tell the story of us. how we met and sparks flew instantly. people would say 'they're the lucky ones'. i used to know my place was a spot next to you. now i'm searching the room for an empty seat. 'cause lately i don't even know what page you're on. oh. a simple complication, miscommunication. has lead to fallout. too many things that i wish you knew. so many walls up i can't break through. now i'm standing alone in a crowded room. and we're not speaking. and i'm dying to know. is it killing you like it's killing me?. and i don't know what to say. since the twist of fate. when it all broke down. and the story of us looks a lot like a tragedy now. next chapter. how'd we end up this way?. se me nervously pulling at my clothes. and trying to look busy. and you're doing your best to avoid me. i'm starting to think one day i'll tell the story of us. how i was losing my mind when i saw you here. but you held your pride like you should've held me. oh. i'm scared 

Remove digits, accented characters, contractions

In [13]:
%%time

genres_df['lyrics'] =genres_df['lyrics'].apply(remove_digits)
genres_df['lyrics'] =genres_df['lyrics'].apply(remove_accented_chars)
genres_df['lyrics'] =genres_df['lyrics'].apply(expand_contractions)

CPU times: user 19.2 s, sys: 62.8 ms, total: 19.3 s
Wall time: 19.3 s


In [14]:
genres_df.iloc[84304]['lyrics']#genres Taylor Swift The Story of Us

'i used to think one day we would tell the story of us. how we met and sparks flew instantly. people would say they are the lucky ones. i used to know my place was a spot next to you. now I am searching the room for an empty seat. because lately i do not even know what page you are on. oh. a simple complication, miscommunication. has lead to fallout. too many things that i wish you knew. so many walls up i can not break through. now I am standing alone in a crowded room. and we are not speaking. and I am dying to know. is it killing you like it is killing me?. and i do not know what to say. since the twist of fate. when it all broke down. and the story of us looks a lot like a tragedy now. next chapter. how did we end up this way?. se me nervously pulling at my clothes. and trying to look busy. and you are doing your best to avoid me. I am starting to think one day I will tell the story of us. how i was losing my mind when i saw you here. but you held your pride like you should have he

I'm going to branch the pre-processing here at this point.  While there is still punctuation.  I think spaCy uses punctuation  for POS and I'm not sure NLTK does.  Want to see the differences.

In [ ]:
!touch genres_midcln_df.csv
genres_df.to_csv('genres_midcln_df.csv', index= False)
!aws s3 cp genres_midcln_df.csv s3://music-lyrics-chain/genres_midcln_df.csv
%time

Remove punct, replace with a space. Remove stop letters 2 characters or less. Also, gensim tokenization.

In [15]:
%%time

genres_df['lyrics'] =genres_df['lyrics'].apply(replace_punctuation)
genres_df['lyrics'] =genres_df['lyrics'].apply(stops_letters)

CPU times: user 40.7 s, sys: 94.6 ms, total: 40.8 s
Wall time: 40.8 s


In [16]:
genres_df.iloc[84304]['lyrics']#genres Taylor Swift The Story of Us

'think day tell story met sparks flew instantly people lucky ones know place spot searching room seat lately know page simple complication lead fallout things wish knew walls break standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy chapter end way nervously pulling clothes trying look busy best avoid starting think day tell story losing mind saw held pride like held scared ending pretending tell miss know heard silence loud standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy looking like contest act like care liked better battles hands lay armor love fight things wish knew story ending soon standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy speaking dying know killing like killing know twist fate going story looks lot like tragedy end'

In [17]:
g_count_df = genres_df.assign(avg_words=genres_df['lyrics'].str.split().str.len())
g_cnt_df = pd.DataFrame((g_count_df), columns = ['song_name', 'genre', 'avg_words'])
g_cnt_df.describe(include='all')

,song_name,genre,avg_words
count,86391,86391,86391.000000
unique,66856,6,NaN
top,Intro,Rock,NaN
freq,50,47408,NaN
mean,NaN,NaN,108.421444
std,NaN,NaN,74.627203
min,NaN,NaN,0.000000
25%,NaN,NaN,61.000000
50%,NaN,NaN,88.000000
75%,NaN,NaN,129.000000


The total number of words went down to 39% of the previous average.  Seems like A LOT.
Decades dataset has a mean of 73 words.

Below: genres_df, before function stops_letters:

    song_name	genre	avg_words
    count	86419	86419	86419.000000
    unique	66869	6	NaN
    top	Intro	Rock	NaN
    freq	50	47409	NaN
    mean	NaN	NaN	274.471389
        std	NaN	NaN	168.492728
        min	NaN	NaN	1.000000
        25%	NaN	NaN	163.000000
        50%	NaN	NaN	232.000000
        75%	NaN	NaN	336.000000
        max	NaN	NaN	3301.000000

Affinity Score - Sentiment Analysis.  This uses AFINN Lexicon, which is for microblogs (tweets).  This may not be the best sentiment lexicon for songs.  spaCy uses a different one.  Another reason to have a branch off of this trunk.

In [18]:
afinn = Afinn()

def get_affinity_scores(lyrics):
    scores = []
    count = 0
    for t in lyrics:
        if len(t) > 0:
            scores.append(afinn.score(t) / len(t))
        else:
            count += 1
            scores.append(0)
    return scores

%time

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 2.86 µs


In [19]:
new_affin = get_affinity_scores(genres_df['lyrics'].tolist())

%time

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.86 µs


In [20]:
genres_df['content_affin'] = new_affin

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


In [ ]:
!touch genres_main_affin_df.csv
genres_df.to_csv('genres_main_affin_df.csv', index= False)
!aws s3 cp genres_main_affin_df.csv s3://music-lyrics-chain/genres_main_affin_df.csv
%time

In [21]:
genres_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,content_affin
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,feel time way knowing fallen leaves night blow...,ENGLISH,Rock,NaN,0.041152
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,baby hold close try understand desire hunger b...,ENGLISH,Rock,NaN,-0.006711
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,days remember promise world warm feel know tru...,ENGLISH,Rock,NaN,0.057047
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,lie mountain coal veins beds dig men axes dig ...,ENGLISH,Rock,NaN,-0.027473
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly wet sand bench clothes stolen ...,ENGLISH,Rock,NaN,0.009615


Normalize Lemmatize

In [22]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
  

def lemmatized_word(text):

    word_tokens = nltk.word_tokenize(text)
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]
    return  " ".join(lemmatized_word) #combine the words into a giant string that vectorizer can accept

[nltk_data] Downloading package punkt to /Users/Gretzky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Gretzky/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
genres_df['vector'] = genres_df['lyrics'].apply(lemmatized_word)

%time

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 2.86 µs


In [24]:
genres_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,content_affin,vector
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,feel time way knowing fallen leaves night blow...,ENGLISH,Rock,NaN,0.041152,feel time way knowing fallen leaf night blowin...
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,baby hold close try understand desire hunger b...,ENGLISH,Rock,NaN,-0.006711,baby hold close try understand desire hunger b...
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,days remember promise world warm feel know tru...,ENGLISH,Rock,NaN,0.057047,day remember promise world warm feel know true...
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,lie mountain coal veins beds dig men axes dig ...,ENGLISH,Rock,NaN,-0.027473,lie mountain coal vein bed dig men ax dig lie ...
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly wet sand bench clothes stolen ...,ENGLISH,Rock,NaN,0.009615,trudging slowly wet sand bench clothes stolen ...


In [25]:
genres_df.iloc[84304]['lyrics']#genres Taylor Swift The Story of Us

'think day tell story met sparks flew instantly people lucky ones know place spot searching room seat lately know page simple complication lead fallout things wish knew walls break standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy chapter end way nervously pulling clothes trying look busy best avoid starting think day tell story losing mind saw held pride like held scared ending pretending tell miss know heard silence loud standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy looking like contest act like care liked better battles hands lay armor love fight things wish knew story ending soon standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy speaking dying know killing like killing know twist fate going story looks lot like tragedy end'

In [26]:
genres_df.iloc[84304]['vector']#genres Taylor Swift The Story of Us

'think day tell story met spark flew instantly people lucky one know place spot searching room seat lately know page simple complication lead fallout thing wish knew wall break standing crowded room speaking dying know killing like killing know twist fate broke story look lot like tragedy chapter end way nervously pulling clothes trying look busy best avoid starting think day tell story losing mind saw held pride like held scared ending pretending tell miss know heard silence loud standing crowded room speaking dying know killing like killing know twist fate broke story look lot like tragedy looking like contest act like care liked better battle hand lay armor love fight thing wish knew story ending soon standing crowded room speaking dying know killing like killing know twist fate broke story look lot like tragedy speaking dying know killing like killing know twist fate going story look lot like tragedy end'

What did this buy me, really?

In [27]:
%%time
# word counts
genres_df['word_count'] = genres_df["lyrics"].apply(lambda x: len(str(x).split(" ")))

# Character counts
genres_df['character_count'] = genres_df["lyrics"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#average word length
genres_df['avg_word_length'] = genres_df['character_count'] / genres_df['word_count']

CPU times: user 1.28 s, sys: 6.85 ms, total: 1.29 s
Wall time: 1.29 s


In [ ]:
!touch genres_affin_plus_df.csv
genres_df.to_csv('genres_affin_plus_df.csv', index= False)
!aws s3 cp genres_affin_plus_df.csv s3://music-lyrics-chain/genres_affin_plus_df.csv
%time

Identify Sentiment, using affinity work from earlier.

In [28]:
def sentiment_check (text):
    polarity_score = TextBlob(text).sentiment.polarity
    genres_df['sent_score'] = polarity_score
    if polarity_score < 0:
        return 'negative'
    elif polarity_score == 0:
        return 'neutral'
    else:
        return 'positive'

In [29]:
%%time
genres_df['sent_label'] = genres_df['lyrics'].apply(sentiment_check)


CPU times: user 39.3 s, sys: 78.8 ms, total: 39.4 s
Wall time: 39.4 s


In [ ]:
# Two additional cleanups.  Found during further EDA.  Remove tiny genres, remove three
#songs that have NaN lyrics.

genres_df.drop(genres_df[genres_df['genre']=='Samba'].index, inplace = True)
genres_df.drop(genres_df[genres_df['genre']=='Sertanejo'].index, inplace = True)
genres_df.drop(genres_df[genres_df['genre']=='Funk Carioca'].index, inplace = True)

genres_df = genres_df.dropna(axis=0, subset=['lyrics'])

In [30]:
genres_df.head(3)

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,content_affin,vector,word_count,character_count,avg_word_length,sent_score,sent_label
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,feel time way knowing fallen leaves night blow...,ENGLISH,Rock,NaN,0.041152,feel time way knowing fallen leaf night blowin...,42,202,4.809524,0.456061,positive
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,baby hold close try understand desire hunger b...,ENGLISH,Rock,NaN,-0.006711,baby hold close try understand desire hunger b...,93,504,5.419355,0.456061,positive
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,days remember promise world warm feel know tru...,ENGLISH,Rock,NaN,0.057047,day remember promise world warm feel know true...,49,250,5.102041,0.456061,positive


In [31]:
genres_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86391 entries, 0 to 86390
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   original_csv     86391 non-null  object 
 1   artist_name      86391 non-null  object 
 2   song_name        86391 non-null  object 
 3   link             86391 non-null  object 
 4   lyrics           86391 non-null  object 
 5   language         86391 non-null  object 
 6   genre            86391 non-null  object 
 7   date             0 non-null      float64
 8   content_affin    86391 non-null  float64
 9   vector           86391 non-null  object 
 10  word_count       86391 non-null  int64  
 11  character_count  86391 non-null  int64  
 12  avg_word_length  86391 non-null  float64
 13  sent_score       86391 non-null  float64
 14  sent_label       86391 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 9.9+ MB


In [32]:
!touch genres_step_3_df.csv
genres_df.to_csv('genres_step_3_df.csv', index= False)
!aws s3 cp genres_step_3_df.csv s3://music-lyrics-chain/genres_step_3_df.csv
%time

upload: ./genres_step_3_df.csv to s3://music-lyrics-chain/genres_step_3_df.csv
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 15.3 µs


In [33]:
genres_df.loc[84304]

original_csv                                              genres_csv
artist_name                                             taylor-swift
song_name                                            The Story Of Us
link                              /taylor-swift/the-story-of-us.html
lyrics             think day tell story met sparks flew instantly...
language                                                     ENGLISH
genre                                                            Pop
date                                                             NaN
content_affin                                              -0.022556
vector             think day tell story met spark flew instantly ...
word_count                                                       151
character_count                                                  781
avg_word_length                                             5.172185
sent_score                                                  0.456061
sent_label                        